# Introduction

This Gaia-alignment pipeline uses the high signal-to-noise drizzled images to calculate offsets to the Gaia reference frame. Therefore, it is necessary to drizzle the individual exposures together before calculating offsets to Gaia. This notebook drizzles HST images, but does not align them to Gaia. The alignment to Gaia is performed within the notebook titled *2) Align Drizzled Images to Gaia*.

This notebook utilizes [TweakReg](https://drizzlepac.readthedocs.io/en/deployment/tweakreg.html) and [AstroDrizzle](https://drizzlepac.readthedocs.io/en/deployment/astrodrizzle.html) to perform an initial alignment the stacking of individual exposures. TweakReg has options to align to an externel reference frame (e.g., Gaia; please see Notebook tutorial [here](https://github.com/spacetelescope/notebooks/blob/master/notebooks/DrizzlePac/align_to_catalogs/align_to_catalogs.ipynb)). However, TweakReg can struggle when there are many cosmic rays and few stars in the FOV. This pipeline was created to assist with aligning to an external reference frame when TweakReg fails.

# Imports

In [ ]:
from glob import glob
import os
import shutil
from astroquery.mast import Observations
from drizzlepac import tweakreg, astrodrizzle
from astropy.table import Table

# Define parent directory
# The working directory is changed later in the Notebook
parent = os.getcwd()

# Download Data

Test HST data is downloaded below. It includes ACS/WFC F606W imaging of the JWST North Ecliptic Pole Time Domain Field. Please see [O'Brien+2024](https://ui.adsabs.harvard.edu/abs/2024arXiv240104944O/abstract) for example use of this data.

In [ ]:
# ### Query observations ###
obs_table = Observations.query_criteria(proposal_pi="Jansen*", proposal_id = 15278, filters = 'F606W')
data_products = Observations.get_product_list(obs_table)

### Download FLC files ###
data_flc = data_products[(data_products['productSubGroupDescription'] == 'FLC') & (data_products['type'] == 'S')]
Observations.download_products(data_flc[:10]) # Only download the first few images

Downloaded files are moved to a directory where the drizzling will take place, so the original files are preserved:

In [ ]:
# Ensure you are in parent directory
os.chdir(parent)

# Specify name of drizzling working directory
wdir = 'drizzled_products_unaligned/'

# Create directory if it does not already exist
if os.path.exists(wdir) == False:
    os.mkdir(wdir)
    
# Copy files to working directory
for og_file in glob('mastDownload/HST/*/*_flc.fits'):
    
    new_path = os.path.join(wdir, os.path.basename(og_file))
    
    print('Moving', og_file, 'to', new_path, '.')
    
    shutil.copyfile(og_file, new_path)
    
# Change location to the drizzling working directory
os.chdir(wdir)
print('\n!! NEW WORKING DIRECTORY:', wdir)

# Drizzle

In [ ]:
# Define individual exposures to drizzle
input_flcs = glob('*_flc.fits')
input_flcs

Align images to each other using TweakReg. For specifics on how to properly run TweakReg, please refer to official TweakReg documentation and tutorials. There will still be an offset with respect to Gaia.

In [ ]:
# Align the images using tweakreg.
tweakreg.TweakReg(input_flcs,
                  searchunits = 'arcseconds',
                  searchrad = 0.1,
                  interactive = False, 
                  shiftfile = True,
                  outshifts = 'shifts.txt',
                  imagefindcfg={'threshold': 300, 'conv_width': 6.0, 'dqbits': 0},
                  sigma = 3.0,
                  nclip = 3,
                  minobj = 10,
                  updatehdr = True, 
                  expand_refcat = True,
                  clean = False)

# Read the final shifts file to ensure the rms offsets are reasonable (typically less than 10 mas)
shift_tab = Table.read('shifts.txt',
                   format='ascii.no_header',
                   names=['file','dx','dy','rot','scale','xrms','yrms'])
formats = ['.2f', '.2f', '.3f', '.5f', '.2f', '.2f']
for i, col in enumerate(shift_tab.colnames[1:]):
    shift_tab[col].format = formats[i]
print(shift_tab)

Drizzle aligned images.

In [ ]:
astrodrizzle.AstroDrizzle(input_flcs,
                          output='jdkq01',
                          final_bits = [1,2,32,64],
                          final_kernel = 'square',
                          final_scale = 0.06,
                          build = False, 
                          combine_type = 'median', 
                          preserve = False,
                          combine_nhigh = 1, 
                          clean = True, 
                          driz_cr_corr = True, 
                          driz_cr_snr = '3.5 3.0',
                          final_wht_type = 'WHT', 
                          runfile='driz_log.txt')